In [7]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split

First we obtain the set of documents.

In [2]:
data = pd.read_csv("train.csv")

documents = data.id.unique()



We perform a small train test split to make sure everything works.

In [4]:
small_train, big_test = train_test_split(documents, test_size=.99)

First we will see if we can correctly predict claims, so we create the training labels as follows.

To each document we associate a vector x such that if the ith word in the document belongs to a claim then x[i]=1 and is otherwise 0.

Here we assume each document has less than 400 words.

In [5]:
def get_claim_indices(document):
    doc = data[data.id==document ]

    doc_claims = doc[doc.discourse_type=='Claim']

    claim_strings = doc_claims.predictionstring.values

    indices = set()

    for string in claim_strings:
        for x in string.split():
            indices.add(int(x))
        
    pre_tensor = []

    i = 0 
    while i<400:
        if i in indices:
            pre_tensor.append(1)
        else: 
            pre_tensor.append(0)
        i+=1

    tensor = tf.constant(pre_tensor)
    return tensor

In [8]:
## build the training labels

y_small_train = []
i = 0
for document in small_train:
    y_small_train.append(get_claim_indices(document))
    

2022-05-24 23:36:08.616322: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [28]:
import os
import tensorflow_hub as hub

import tensorflow.keras as keras

import tensorflow_text as text

import bert

from transformers import TFBertModel, PreTrainedTokenizer, PreTrainedTokenizerBase

from tensorflow.keras import layers


import functools

Now we load the tokenizer and Bert encoder.

In [29]:
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
save_path = "bert_base_uncased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)


('bert_base_uncased/tokenizer_config.json',
 'bert_base_uncased/special_tokens_map.json',
 'bert_base_uncased/vocab.txt',
 'bert_base_uncased/added_tokens.json')

In [45]:
tokenizer = BertTokenizer("bert_base_uncased/vocab.txt", model_max_length=512, lowercase=True)

tokenizer.vocab_size

30522

In [31]:
encoder = TFBertModel.from_pretrained("bert-base-uncased")

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Now we use the tokenizer and encoder to preprocess the essays.

In [47]:
X_small_train = []
X_tokens = []
for document in small_train:
    
    with open('train/'+str(document)+'.txt') as f:
        lines = f.read()
    
    
    tokens = tokenizer(lines, return_tensors = 'tf', truncation=True ,pad_to_max_length=True)
    
    print("tokens made")
    X_small_train.append(encoder(tokens))
    print("bert ran")
    

tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
bert ran
tokens made
b

In [140]:
X = [x['last_hidden_state'].reshape(-1,1) for x in X_small_train]


In [54]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

X_small_train[0]['last_hidden_state'].reshape(1,768*512)

<tf.Tensor: shape=(1, 393216), dtype=float32, numpy=
array([[ 1.7543536e-01, -3.4123439e-01, -4.4638878e-01, ...,
         5.2832294e-01, -4.5026094e-04, -1.7864566e-01]], dtype=float32)>

In [58]:
model=keras.Sequential()

model.add(layers.Dense(512))

In [137]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (1, 512)                  201327104 
                                                                 
Total params: 201,327,104
Trainable params: 201,327,104
Non-trainable params: 0
_________________________________________________________________


In [124]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss=keras.losses.mean_squared_error)

In [139]:
X[0].reshape(-1,1)

<tf.Tensor: shape=(393216, 1), dtype=float32, numpy=
array([[ 1.7543536e-01],
       [-3.4123439e-01],
       [-4.4638878e-01],
       ...,
       [ 5.2832294e-01],
       [-4.5026094e-04],
       [-1.7864566e-01]], dtype=float32)>

In [150]:
history = model.fit(
    X,
    y_batches,
    batch_size=10,
    epochs=2
)

ValueError: Data cardinality is ambiguous:
  x sizes: 393216, 393216, 393216, 328704, 393216, 393216, 393216, 393216, 340224, 393216, 393216, 175872, 393216, 393216, 393216, 250368, 214272, 301824, 393216, 261120, 269568, 393216, 307968, 284928, 305664, 346368, 393216, 254976, 323328, 158208, 393216, 393216, 188160, 172032, 138240, 393216, 215040, 352512, 393216, 188160, 393216, 393216, 393216, 393216, 393216, 393216, 393216, 223488, 393216, 393216, 393216, 370176, 393216, 147456, 310272, 172800, 321024, 139776, 393216, 237312, 393216, 158208, 393216, 370944, 230400, 393216, 377088, 393216, 393216, 393216, 393216, 361728, 388608, 277248, 393216, 393216, 218880, 393216, 393216, 200448, 211200, 365568, 163584, 189696, 393216, 380160, 251904, 393216, 154368, 393216, 281088, 393216, 393216, 301824, 201216, 393216, 240384, 388608, 393216, 172032, 337152, 393216, 262656, 275712, 207360, 371712, 350976, 364800, 151296, 393216, 285696, 375552, 393216, 393216, 325632, 393216, 256512, 207360, 34

In [146]:
y_batches = []
for label in y:
    new_batch = []
    count = 0
    while count<10:
        new_batch.append(label)
        count+=1
    y_batches.append(new_batch)

In [147]:
y_batches

[[[[<tf.Tensor: shape=(400,), dtype=int32, numpy=
    array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 0, 